In [ ]:
!pip install sagemaker -q -U

In [ ]:
!pip install datasets -q -U

In [4]:
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::905847418383:role/service-role/AmazonSageMaker-ExecutionRole-20211005T160629
sagemaker bucket: sagemaker-us-east-1-905847418383
sagemaker session region: us-east-1


In [7]:
!wget https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/dev-v0.1.json

--2021-10-06 14:12:18--  https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/dev-v0.1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 458494 (448K) [text/plain]
Saving to: ‘dev-v0.1.json.1’

dev-v0.1.json.1     100%[===================>] 447.75K  --.-KB/s    in 0.005s  

2021-10-06 14:12:18 (92.9 MB/s) - ‘dev-v0.1.json.1’ saved [458494/458494]



In [8]:
!wget https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/train-v0.1.json

--2021-10-06 14:12:29--  https://raw.githubusercontent.com/TQuad/turkish-nlp-qa-dataset/master/train-v0.1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3078146 (2.9M) [text/plain]
Saving to: ‘train-v0.1.json.1’

train-v0.1.json.1   100%[===================>]   2.94M  --.-KB/s    in 0.01s   

2021-10-06 14:12:29 (251 MB/s) - ‘train-v0.1.json.1’ saved [3078146/3078146]



In [13]:
bucket = sess.default_bucket()

In [42]:
!aws s3 cp data/train.json s3://$bucket/qa-tr/data/train.json
!aws s3 cp data/val.json s3://$bucket/qa-tr/data/val.json

upload: data/train.json to s3://sagemaker-us-east-1-905847418383/qa-tr/data/train.json
upload: data/val.json to s3://sagemaker-us-east-1-905847418383/qa-tr/data/val.json


In [46]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={
    'model_name_or_path': 'dbmdz/bert-base-turkish-uncased',
#     'dataset_name':'squad',
    'train_file': '/opt/ml/input/data/train/train.json',
    'validation_file': '/opt/ml/input/data/val/val.json',
    'do_train': True,
    'do_eval': True,
    'fp16': True,
    'per_device_train_batch_size': 4,
    'per_device_eval_batch_size': 4,
    'num_train_epochs': 1,
    'max_seq_length': 384,
    'max_steps': 100,
    'pad_to_max_length': True,
    'doc_stride': 128,
    'output_dir': '/opt/ml/model'
}

# configuration for running training on smdistributed Data Parallel
# distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

# git configuration to download our fine-tuning script
git_config = {'repo': 'https://github.com/marshmellow77/transformers.git','branch': 'v4.6.0-release'}

# instance configurations
instance_type = 'ml.p3.16xlarge'
instance_count = 1
volume_size = 200

In [47]:
huggingface_estimator = HuggingFace(entry_point='run_qa.py',
                                    source_dir='./examples/pytorch/question-answering',
                                    git_config=git_config,
#                                     metric_definitions=metric_definitions,
                                    instance_type=instance_type,
                                    instance_count=instance_count,
                                    volume_size=volume_size,
                                    role=role,
                                    transformers_version='4.6',
                                    pytorch_version='1.7',
                                    py_version='py36',
#                                     distribution= distribution,
                                    hyperparameters= hyperparameters,
                                    disable_profiler=True)

In [48]:
huggingface_estimator.fit({'train': 's3://sagemaker-us-east-1-905847418383/qa-tr/data/', 'val': 's3://sagemaker-us-east-1-905847418383/qa-tr/data/'})

2021-10-07 08:43:29 Starting - Starting the training job...
2021-10-07 08:43:34 Starting - Launching requested ML instances............
2021-10-07 08:45:34 Starting - Preparing the instances for training......
2021-10-07 08:46:56 Downloading - Downloading input data......
2021-10-07 08:47:35 Training - Downloading the training image............
2021-10-07 08:50:00 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-10-07 08:50:00,828 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-10-07 08:50:00,907 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-10-07 08:50:02,377 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-10-07 08:50:02,791 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/py

UnexpectedStatusException: Error for Training job huggingface-pytorch-training-2021-10-07-08-43-23-394: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python3.6 run_qa.py --do_eval True --do_train True --doc_stride 128 --fp16 True --max_seq_length 384 --max_steps 100 --model_name_or_path dbmdz/bert-base-turkish-uncased --num_train_epochs 1 --output_dir /opt/ml/model --pad_to_max_length True --per_device_eval_batch_size 4 --per_device_train_batch_size 4 --train_file /opt/ml/input/data/train/train.json --validation_file /opt/ml/input/data/val/val.json"
0 tables [00:00, ? tables/s]                            0 tables [00:00, ? tables/s]                            [INFO|file_utils.py:1532] 2021-10-07 08:50:11,373 >> https://huggingface.co/dbmdz/bert-base-turkish-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpn03ug29m
Downloading:   0%|          | 0.00/385 [00: